In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [10]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band8
  atmpro = mls
  band = 8
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, -1, 0]
  ng_refs = [3, 2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 1900
  vmin = 1380
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.55, 0.55, 0.85), (0.85, 0.85), (0, 0.3, 0.55)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.118864                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.139879
0.000750      2.0 -0.117997                  2.0 -0.139078
0.001052      3.0 -0.115269                  3.0 -0.137114
0.001476      4.0 -0.112568                  4.0 -0.135132
0.002070      5.0 -0.110018                  5.0 -0.133150
0.002904      6.0 -0.107682                  6.0 -0.131208
0.004074      7.0 -0.105689                  7.0 -0.129372
0.005714      8.0 -0.104155                  8.0 -0.127650
0.008015      9.0 -0.103292                  9.0 -0.126195
0.011243     10.0 -0.103328                 10.0 -0.125073
0.015771     11.0 -0.104607                 11.0 -0.124488
0.022122     12.0 -0.107622                 12.0 -0.124718
0.031031     13.0 -0.113104                 13.0 -0.126088
0.043528     14.0 -0.121142                 14.0 -0.128376
0.061057     15.0 -0.119447                 15.0 -0.122709
0.085645     16.0 -0.097580                 16.0 -0.100923
0.120136     17.0 -0.063157                 17.0 -0.067139
0.168516     18.0 -0.022470                 18.0 -0.025150
0.236378     19.0  0.025437                 19.0  0.025721
0.331549     20.0  0.083077                 20.0  0.087089
0.465100     21.0  0.157618                 21.0  0.163223
0.652400     22.0  0.248863                 22.0  0.248256
0.915100     23.0  0.252229                 23.0  0.248887
1.283650     24.0  0.178745                 24.0  0.173318
1.800600     25.0  0.117896                 25.0  0.112010
2.525700     26.0  0.070128                 26.0  0.069317
3.542800     27.0  0.044457                 27.0  0.047661
4.969550     28.0  0.031931                 28.0  0.034740
6.970850     29.0  0.024893                 29.0  0.026147
9.778100     30.0  0.020104                 30.0  0.020807
13.715850    31.0  0.016195                 31.0  0.017708
19.239350    32.0  0.012686                 32.0  0.014771
26.987250    33.0  0.009528                 33.0  0.011493
37.855300    34.0  0.006816                 34.0  0.008571
53.100050    35.0  0.001700                 35.0  0.002869
73.887500    36.0 -0.004887                 36.0 -0.005248
97.662500    37.0 -0.009644                 37.0 -0.011021
121.437500   38.0 -0.010153                 38.0 -0.011295
145.212500   39.0 -0.006662                 39.0 -0.007082
168.987500   40.0 -0.002946                 40.0 -0.003042
192.762500   41.0  0.002896                 41.0  0.002907
216.537500   42.0  0.016067                 42.0  0.019479
240.312500   43.0  0.035440                 43.0  0.047954
264.087500   44.0  0.060125                 44.0  0.062789
287.862500   45.0  0.088458                 45.0  0.063097
311.637500   46.0  0.117278                 46.0  0.089768
335.412500   47.0  0.142320                 47.0  0.134831
359.187500   48.0  0.157582                 48.0  0.162399
382.962500   49.0  0.157965                 49.0  0.158523
406.737500   50.0  0.157831                 50.0  0.154041
430.512500   51.0  0.154624                 51.0  0.157155
454.287500   52.0  0.148156                 52.0  0.162761
478.062500   53.0  0.138975                 53.0  0.161147
501.837500   54.0  0.127845                 54.0  0.150121
525.612500   55.0  0.115959                 55.0  0.129347
549.387500   56.0  0.103661                 56.0  0.106132
573.162500   57.0  0.091681                 57.0  0.085426
596.937500   58.0  0.080798                 58.0  0.070631
620.712500   59.0  0.072027                 59.0  0.062358
644.487500   60.0  0.068042                 60.0  0.060986
668.262500   61.0  0.071532                 61.0  0.066934
692.037500   62.0  0.079755                 62.0  0.076791
715.812500   63.0  0.078998                 63.0  0.075270
739.587500   64.0  0.059676                 64.0  0.055988
76

# Fluxes by Layer

CRD                          CLIRAD                \
                         flug       fldg     fnetg       flug          fldg   
pressure     level                                                            
0.000000e+00 1      -7.715035   0.000000 -7.715035        NaN           NaN   
1.000000e-08 1            NaN        NaN       NaN  -7.574985  1.875993e-09   
6.244000e-04 2      -7.715048   0.000004 -7.715044  -7.574998  2.836569e-06   
8.759000e-04 3      -7.715053   0.000006 -7.715047  -7.575004  3.993923e-06   
1.228600e-03 4      -7.715060   0.000009 -7.715052  -7.575011  5.665512e-06   
1.723400e-03 5      -7.715071   0.000012 -7.715059  -7.575021  8.080453e-06   
2.417400e-03 6      -7.715085   0.000017 -7.715067  -7.575036  1.156749e-05   
3.390900e-03 7      -7.715105   0.000025 -7.715080  -7.575056  1.660031e-05   
4.756500e-03 8      -7.715132   0.000035 -7.715097  -7.575084  2.385732e-05   
6.672000e-03 9      -7.715171   0.000050 -7.715121  -7.575124  3.431395e-05   
9.358900e-03 10     -7.715225   0.000071 -7.715154  -7.575179  4.935565e-05   
1.312780e-02 11     -7.715301   0.000101 -7.715200  -7.575256  7.095645e-05   
1.841450e-02 12     -7.715408   0.000142 -7.715265  -7.575365  1.018927e-04   
2.583020e-02 13     -7.715560   0.000200 -7.715360  -7.575519  1.460386e-04   
3.623230e-02 14     -7.715777   0.000278 -7.715499  -7.575737  2.087504e-04   
5.082340e-02 15     -7.716093   0.000385 -7.715709  -7.576048  2.979821e-04   
7.129060e-02 16     -7.716545   0.000547 -7.715998  -7.576485  4.371364e-04   
1.000000e-01 17     -7.717160   0.000830 -7.716330  -7.577074  6.831244e-04   
1.402710e-01 18     -7.717967   0.001336 -7.716631  -7.577841  1.129572e-03   
1.967600e-01 19     -7.718988   0.002207 -7.716782  -7.578801  1.921581e-03   
2.759970e-01 20     -7.720208   0.003665 -7.716543  -7.579927  3.288716e-03   
3.871000e-01 21     -7.721494   0.006045 -7.715449  -7.581068  5.576612e-03   
5.431000e-01 22     -7.722380   0.009843 -7.712537  -7.581758  9.282377e-03   
7.617000e-01 23     -7.721634   0.015542 -7.706093  -7.580915  1.486853e-02   
1.068500e+00 24     -7.718865   0.021939 -7.696926  -7.578204  2.120368e-02   
1.498800e+00 25     -7.714936   0.027121 -7.687815  -7.574380  2.621538e-02   
2.102400e+00 26     -7.710141   0.030756 -7.679385  -7.569705  2.954983e-02   
2.949000e+00 27     -7.705111   0.032759 -7.672352  -7.564778  3.157491e-02   
4.136600e+00 28     -7.700015   0.033917 -7.666098  -7.559751  3.325411e-02   
5.802500e+00 29     -7.694784   0.034987 -7.659797  -7.554560  3.491939e-02   
8.139200e+00 30     -7.689338   0.036432 -7.652907  -7.549067  3.666471e-02   
1.141700e+01 31     -7.683626   0.038525 -7.645101  -7.543166  3.884332e-02   
1.601470e+01 32     -7.677654   0.041374 -7.636280  -7.536761  4.208384e-02   
2.246400e+01 33     -7.671554   0.044965 -7.626588  -7.529920  4.652838e-02   
3.151050e+01 34     -7.665589   0.049210 -7.616378  -7.522836  5.176116e-02   
4.420010e+01 35     -7.659923   0.053791 -7.606133  -7.515631  5.744187e-02   
6.200000e+01 36     -7.658706   0.056159 -7.602547  -7.512804  6.066549e-02   
8.577500e+01 37     -7.671281   0.054969 -7.616312  -7.524985  5.806413e-02   
1.095500e+02 38     -7.698366   0.054895 -7.643471  -7.553712  5.575051e-02   
1.333250e+02 39     -7.735633   0.063566 -7.672066  -7.593966  6.419032e-02   
1.571000e+02 40     -7.776711   0.085883 -7.690828  -7.638301  8.857769e-02   
1.808750e+02 41     -7.822204   0.123079 -7.699125  -7.687199  1.289068e-01   
2.046500e+02 42     -7.881532   0.190563 -7.690969  -7.750867  2.007633e-01   
2.284250e+02 43     -7.986523   0.340805 -7.645718  -7.861412  3.661741e-01   
2.522000e+02 44     -8.151701   0.605794 -7.545907  -8.033205  6.730358e-01   
2.759750e+02 45     -8.389547   1.012971 -7.376576  -8.280023  1.096705e+00   
2.997500e+02 46     -8.709339   1.581890 -7.127449  -8.609942  1.604343e+00   
3.235250e+02 47     -9.116981   2.319824 -6.797157  -9.023511  2.270754e+00 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')